In [10]:
import sys
import getpass
import re
from functools import partial
from tqdm import tqdm
import uuid

user = getpass.getuser()
sys.dont_write_bytecode = True

sys.path.insert(0, '/Users/{}/Box/DataViz Projects/Utility Code'.format(user))

from utils_io import *  # initial imports
reload_module('utils_io')  # reload after update to utils_io.py
from utils_io import *  # reload reflects updates

#geo imports
import geopandas as gp
import folium
from geopandas.tools import geocode
from geopy.geocoders import GoogleV3
from geopy.extra.rate_limiter import RateLimiter
from geopy import Point

## Match un-geocoded projects by APN

-[ParcelAtlas v24 (BASIS)](https://data.bayareametro.gov/Cadastral/ParcelAtlas-v24-BASIS-/8hp3-7hq6)

In [2]:
parcels_id = '8hp3-7hq6'

In [4]:
#Pull Sonoma County Parcels from Socrata
query_args = {'where': "county = 'Santa Clara'"}
parcels_santa_clara = pull_df_from_socrata(parcels_id,query_args=query_args)

pulling data in 22 chunks of 22000 rows each
pulling chunk 0
pulling chunk 1
pulling chunk 2
pulling chunk 3
pulling chunk 4
pulling chunk 5
pulling chunk 6
pulling chunk 7
pulling chunk 8
pulling chunk 9
pulling chunk 10
pulling chunk 11
pulling chunk 12
pulling chunk 13
pulling chunk 14
pulling chunk 15
pulling chunk 16
pulling chunk 17
pulling chunk 18
pulling chunk 19
pulling chunk 20
pulling chunk 21
took 1.8777 minutes


In [21]:
parcels_santa_clara.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463807 entries, 0 to 463806
Data columns (total 46 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   objectid    463807 non-null  float64
 1   apn         463807 non-null  object 
 2   apn2        0 non-null       float64
 3   state       463807 non-null  object 
 4   county      463807 non-null  object 
 5   fips        463807 non-null  object 
 6   sit_hse_nu  440144 non-null  object 
 7   sit_dir     29732 non-null   object 
 8   sit_str_na  444934 non-null  object 
 9   sit_str_sf  435233 non-null  object 
 10  sit_full_s  444935 non-null  object 
 11  sit_city    450034 non-null  object 
 12  sit_state   450724 non-null  object 
 13  sit_zip     450504 non-null  object 
 14  sit_zip4    437791 non-null  object 
 15  land_value  463807 non-null  float64
 16  impr_value  463807 non-null  float64
 17  tot_value   463807 non-null  float64
 18  assmt_year  463807 non-null  float64
 19  re

In [22]:
parcels_santa_clara[['apn','sit_city']].sample(n=35)

,apn,sit_city
395588,35903010,CUPERTINO
366718,37329030,SAN JOSE
50322,67678033,SAN JOSE
21452,78372013,GILROY
143462,48172010,SAN JOSE
290812,26961003,SANTA CLARA
322335,30706018,SAN JOSE
232706,23037039,SAN JOSE
356019,20514034,SUNNYVALE
46343,67823038,SAN JOSE


In [119]:
#Pull un-geocoded sonoma projects from Socrata
apr_id = 'briv-ikjp'
query_args = {'where': "cnty_name = 'SANTA CLARA'"}

In [130]:
sc_projects = pull_df_from_socrata(apr_id,query_args=query_args)

pulling data in 1 chunks of 5146 rows each
pulling chunk 0
took 3.7183 seconds


In [131]:
sc_projects['mtc_geocode_type'].unique()

array(['ROOFTOP', 'APPROXIMATE', 'GEOMETRIC_CENTER', 'RANGE_INTERPOLATED',
       'NOT_GEOCODED'], dtype=object)

In [132]:
sc_projects[['mtc_id','jurs_name','apn','mtc_geocode_type']][sc_projects['mtc_geocode_type'].isin(['NOT_GEOCODED',
                                                                                      'APPROXIMATE',
                                                                                     'GEOMETRIC_CENTER'])].sample(n=30)

,mtc_id,jurs_name,apn,mtc_geocode_type
1559,22cc2ddb-7c72-4dbb-903a-a3f40f363ad9,LOS ALTOS,189 49 011,NOT_GEOCODED
2015,fd0f9ef2-a5dc-4a06-901c-1e4d29f4e578,SAN JOSE,26105043,APPROXIMATE
729,f2cc09da-fa36-4a15-a593-5320ffe4d87f,SANTA CLARA COUNTY,27712059,APPROXIMATE
1747,f86f31f3-d976-4580-b8e1-a13f15fbfdda,SAN JOSE,23703064,NOT_GEOCODED
2865,f751855a-4001-496b-9c21-e03f7ad7e348,SANTA CLARA COUNTY,82517028,GEOMETRIC_CENTER
269,d714a22e-d394-4884-a894-4911827d2a6c,SUNNYVALE,20565043,GEOMETRIC_CENTER
281,046ce6a7-5b27-4e05-949c-f301dd2136ec,SAN JOSE,25406042,GEOMETRIC_CENTER
691,5c322165-44be-4f70-8858-aa64941a40a4,SANTA CLARA COUNTY,55815039,APPROXIMATE
695,e7362144-6f9b-45a2-82ff-5afd1e0585e2,SANTA CLARA COUNTY,53226034,APPROXIMATE
260,328714bd-4501-4561-814c-f9f9fed020ab,SUNNYVALE,20565053,GEOMETRIC_CENTER


## Format APNs to match Santa Clara County pattern

In [133]:
#Subset APNs that match the county pattern
sc_regex_ctny_pattern = r'\d{8}'
sc_apn_reg = sc_projects[['apn']][sc_projects['apn'].str.contains(sc_regex_ctny_pattern,na=False)]

In [134]:
#Create a column with formatted apn, setting it to the first correctly mattched pattern 
#if multiple apns listed in column
sc_regex_extract_ctny_pattern = r'(^\d{8})'
sc_apn_reg['mtc_apn_fmt'] = sc_apn_reg['apn'].str.extract(sc_regex_extract_ctny_pattern)

In [135]:
#update the main dataframe with updated values
sc_projects['mtc_apn_fmt'].update(sc_apn_reg['mtc_apn_fmt'])

In [136]:
sc_projects[sc_projects['mtc_apn_fmt'].isnull()].shape

(355, 75)

In [49]:
sc_projects[['mtc_id','apn','mtc_apn_fmt']][sc_projects['mtc_apn_fmt'].isnull()].sample(n=25)

,mtc_id,apn,apn_fmt
2384,99dd5af6-bbfd-4ad5-bc21-2575cb07b5a6,230-16-002,NaN
2263,628c843d-a5d8-4670-8e88-01e405b2dbf4,NaN,NaN
4203,c7e3631f-a7c9-4a17-9b24-c1615b335613,269-12-017,NaN
358,78be9a96-d870-4008-bc61-85d92a826e43,808 57 024,NaN
2352,7b844ddf-bb54-4901-85cf-0a1f738c46be,220-20-048,NaN
824,4733907c-f258-453e-9c5a-2e41450f108d,8821072,NaN
2276,f4134040-6b56-4992-b7c3-5157d324a083,NaN,NaN
3141,672cdcb8-9066-4e93-a4f9-ac41623fdfff,783-52-029,NaN
3535,62c0413c-0e3a-4ac1-9275-866a5ac2a8ba,189-32-085,NaN
4158,0672d9fd-c066-449a-b717-05c20f44c0cc,216-33-040,NaN


In [137]:
sc_regex = r'-| '

In [138]:
#Subset apns that do not match 8 digit, no space or dash pattern
sc_projects_badapn = sc_projects[['apn']][sc_projects['apn'].str.contains(sc_regex,na=False)]

In [139]:
#Replace apns with spaces or dash pattern with no space or dash pattern
sc_projects_badapn['apn'] = sc_projects_badapn['apn'].replace(sc_regex,'',regex=True)

In [140]:
#Create a column with formatted apn, setting it to the first correctly mattched pattern 
#if multiple apns listed in column
sc_projects_badapn['mtc_apn_fmt'] = sc_projects_badapn['apn'].str.extract(sc_regex_extract_ctny_pattern)

In [142]:
#Update main dataframe with updated projects 
sc_projects['mtc_apn_fmt'].update(sc_projects_badapn['mtc_apn_fmt'])

In [100]:
sc_projects[sc_projects['apn_fmt'].isnull()].shape

(431, 76)

In [101]:
sc_projects[['mtc_id','apn','apn_fmt']][sc_projects['apn_fmt'].isnull()].sample(n=35)

,mtc_id,apn,apn_fmt
4527,8d2f1865-adbe-45f1-b7c3-3b9f22739927,-3929,NaN
781,c0baaf3a-d40d-41f9-aefa-a925d94253a2,2624037,NaN
4525,df1b1258-f707-4c97-a546-bded0374bd5b,-3927,NaN
4448,84d42891-39ad-4a0a-b8ba-e6a47d222786,-3700,NaN
4535,c77eed8d-abb3-4da8-9ad3-b6cb618e44bb,-3937,NaN
4482,12dfe261-8f4b-4d70-b73e-ec8a1119f45e,-3820,NaN
2821,6eee7fdf-32e6-4f3e-921b-b390487633a7,NaN,NaN
2787,944c97a1-d179-4885-83cf-6d96ea20a8ee,NaN,NaN
4476,0e4adfdc-e33b-4600-ad82-9cbd8f10a48c,-3728,NaN
2805,efc0ee5c-46b2-4756-9fdf-a5b831608244,NaN,NaN


In [143]:
#Subset projects with APNs with length less than 8
sc_projects_sevenapn = sc_projects[['apn']][sc_projects['apn'].str.len() == 7]

In [144]:
sc_projects_sevenapn.shape

(76, 1)

In [145]:
#prepend 0 to beginning of apn 
sc_projects_sevenapn['mtc_apn_fmt'] = '0' + sc_projects_sevenapn['apn']

In [146]:
#Update main dataframe with updated apns
sc_projects['mtc_apn_fmt'].update(sc_projects_sevenapn['mtc_apn_fmt'])

In [148]:
sc_projects[sc_projects['mtc_apn_fmt'].isnull()].shape

(355, 75)

In [149]:
cols = ['mtc_id','apn','mtc_apn_fmt','mtc_address_full','mtc_geocode_type']
sc_projects[cols][sc_projects['apn'].str.len() < 7]

,mtc_id,apn,mtc_apn_fmt,mtc_address_full,mtc_geocode_type
324,75ec7975-4f14-4ed8-a6cf-a736115ce37c,-4015,NaN,"845 JULIAN TR 3, SUNNYVALE, CA , USA",ROOFTOP
325,9dd475c4-1b32-41a3-a7da-8246d6efaf30,-4014,NaN,"845 JULIAN TR 2, SUNNYVALE, CA , USA",ROOFTOP
326,c88fe391-68d4-4209-9c87-139a269cf487,-4013,NaN,"845 JULIAN TR 1, SUNNYVALE, CA , USA",ROOFTOP
327,48c39735-ff23-4905-b403-690a2f5d9373,-4012,NaN,"842 GRIDLEY TR 4, SUNNYVALE, CA , USA",RANGE_INTERPOLATED
328,bb2279a5-14f1-43bc-80bd-0ba8a2e9be60,-4011,NaN,"842 GRIDLEY TR 3, SUNNYVALE, CA , USA",RANGE_INTERPOLATED
...,...,...,...,...,...
1999,e2f256ca-2cd5-4a94-a337-62fd7c35484e,-3669,NaN,"905 DE GUIGNE, SUNNYVALE, CA , USA",ROOFTOP
2000,f64ceb5c-d337-41a6-9796-8658327a1b2d,-3668,NaN,"905 DE GUIGNE, SUNNYVALE, CA , USA",ROOFTOP
2001,23f4bd96-1e3c-4816-bc32-1933cfedc31d,-3667,NaN,"905 DE GUIGNE, SUNNYVALE, CA , USA",ROOFTOP
2002,a6d959fd-f736-48ca-a7b9-7018ff3d74f6,-3666,NaN,"905 DE GUIGNE, SUNNYVALE, CA , USA",ROOFTOP


In [150]:
#Upsert apn updates to socrata
upsert_df_socrata(sc_projects[['mtc_id','mtc_apn_fmt']],apr_id)

upserting data to Socrata in 1 chunks of 5146 rows each
upserted chunk 0
data upserted to briv-ikjp
took 3.208 seconds


## Join to parcels by apn

In [173]:
sc_projects.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5146 entries, 0 to 5145
Data columns (total 75 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   mtc_id                       5146 non-null   object 
 1   mtc_type                     5103 non-null   object 
 2   mtc_year                     5146 non-null   float64
 3   mtc_vlow_income_dr           46 non-null     float64
 4   mtc_vlow_income_ndr          138 non-null    float64
 5   mtc_vlow_tot                 5146 non-null   float64
 6   mtc_low_income_dr            75 non-null     float64
 7   mtc_low_income_ndr           53 non-null     float64
 8   mtc_low_tot                  5146 non-null   float64
 9   mtc_mod_income_dr            206 non-null    float64
 10  mtc_mod_income_ndr           427 non-null    float64
 11  mtc_mod_tot                  5146 non-null   float64
 12  mtc_above_mod_income         4313 non-null   float64
 13  mtc_total_units   

In [174]:
#Subset projects to required fields
sc_projects_sub = sc_projects[['mtc_id','apn','mtc_apn_fmt','mtc_mapped','mtc_geocode_type']].copy()

In [175]:
sc_projects_sub

,mtc_id,apn,mtc_apn_fmt,mtc_mapped,mtc_geocode_type
0,a832e543-8f3f-4b87-adc1-c68c7b18e717,20934023,20934023,True,ROOFTOP
1,39c5da24-7e33-45a7-aa4b-d91f155c4395,20934023,20934023,True,ROOFTOP
2,7b8022ce-354e-400a-870f-19aeea7f082e,20934023,20934023,True,ROOFTOP
3,af47b081-be82-4fa7-a9a6-cf351ea5e131,20934023,20934023,True,ROOFTOP
4,dab1cb4d-6069-4073-a055-1f89c8e45620,20934023,20934023,True,ROOFTOP
...,...,...,...,...,...
5141,7a57f8c6-a19c-40e1-95cc-72c7cefe3222,808 57 008,80857008,True,ROOFTOP
5142,9885b44e-a00c-4339-8daf-ef0491e1c01c,808 57 007,80857007,True,ROOFTOP
5143,d7b52cc4-7bf0-4967-aa62-13227ff8de17,808 57 006,80857006,True,ROOFTOP
5144,bbe2919f-8e39-44a9-ad11-87c2c8590f6b,808 57 005,80857005,True,ROOFTOP


In [176]:
sc_projects_parcels = pd.merge(sc_projects_sub,
                                  parcels_santa_clara,
                                  left_on ='mtc_apn_fmt',
                                  right_on='apn',
                                  how='left',
                              indicator=True)

In [177]:
sc_projects_parcels[sc_projects_parcels['_merge'] == 'both'].shape

(3850, 52)

In [199]:
sc_projects_parcels[(sc_projects_parcels['mtc_mapped'] == False)].shape

(103, 52)

In [181]:
sc_projects_parcels[(sc_projects_parcels['_merge'] == 'both') & (sc_projects_parcels['mtc_mapped'] == False)]

,mtc_id,apn_x,mtc_apn_fmt,mtc_mapped,mtc_geocode_type,objectid,apn_y,apn2,state,county,...,minx,miny,maxx,maxy,attdate,version,quantarium,gpid,geometry,_merge
680,b05ece47-5fa7-4dbb-a63b-7127fb8f8c39,70123029,70123029,False,NOT_GEOCODED,5462202.0,70123029,NaN,CA,Santa Clara,...,-121.843087,37.218323,-121.841833,37.219523,20190815,7/,20589154,-121.84246:+37.21892:0001,"{'type': 'MultiPolygon', 'coordinates': [[[[-1...",both
703,ec55d549-5da4-4b99-9d52-60522d065687,42103033,42103033,False,NOT_GEOCODED,5606054.0,42103033,NaN,CA,Santa Clara,...,-121.937544,37.256632,-121.937340,37.257004,20190815,7/,20405935,-121.93744:+37.25682:0001,"{'type': 'MultiPolygon', 'coordinates': [[[[-1...",both
704,d22de640-fa35-4b90-80b7-09b37fcecce6,41929034,41929034,False,NOT_GEOCODED,5595889.0,41929034,NaN,CA,Santa Clara,...,-121.926774,37.251735,-121.926233,37.251946,20190815,7/,20404161,-121.92650:+37.25184:0001,"{'type': 'MultiPolygon', 'coordinates': [[[[-1...",both
765,4154a0aa-1c50-4426-b26c-e9544dc71bb5,20564029,20564029,False,NOT_GEOCODED,5692948.0,20564029,NaN,CA,Santa Clara,...,-122.008685,37.386423,-122.008610,37.386524,NaN,7/,NaN,-122.00865:+37.38647:0001,"{'type': 'MultiPolygon', 'coordinates': [[[[-1...",both
778,6c1bba4c-bac0-4481-bc9f-6c4e6615e8d4,20564012,20564012,False,NOT_GEOCODED,5691633.0,20564012,NaN,CA,Santa Clara,...,-122.007477,37.385554,-122.007353,37.385612,NaN,7/,NaN,-122.00741:+37.38558:0001,"{'type': 'MultiPolygon', 'coordinates': [[[[-1...",both
936,bd675f1a-d590-4be4-9254-5cd98a1e9dd2,42106046,42106046,False,NOT_GEOCODED,5603959.0,42106046,NaN,CA,Santa Clara,...,-121.935280,37.257552,-121.935072,37.257940,20190815,7/,20406188,-121.93518:+37.25775:0001,"{'type': 'MultiPolygon', 'coordinates': [[[[-1...",both
937,e56d9a38-680a-4746-a0db-5c79fac82101,42106046,42106046,False,NOT_GEOCODED,5603959.0,42106046,NaN,CA,Santa Clara,...,-121.935280,37.257552,-121.935072,37.257940,20190815,7/,20406188,-121.93518:+37.25775:0001,"{'type': 'MultiPolygon', 'coordinates': [[[[-1...",both
962,d6c1d1e5-6e32-489c-ba93-7f1610f14769,70607020,70607020,False,NOT_GEOCODED,5398807.0,70607020,NaN,CA,Santa Clara,...,-121.801010,37.240273,-121.788344,37.251014,20190815,7/,20593387,-121.79468:+37.24564:0001,"{'type': 'MultiPolygon', 'coordinates': [[[[-1...",both
1107,591ddf7d-fd99-4552-a100-b47f891168a4,47723043,47723043,False,NOT_GEOCODED,5492622.0,47723043,NaN,CA,Santa Clara,...,-121.863270,37.305294,-121.861587,37.306463,20190815,7/,20452908,-121.86243:+37.30588:0001,"{'type': 'MultiPolygon', 'coordinates': [[[[-1...",both
1128,a5e0ebe5-53ef-4143-9742-1ceac528ed37,46701008,46701008,False,NOT_GEOCODED,5544899.0,46701008,NaN,CA,Santa Clara,...,-121.892793,37.339251,-121.892271,37.339600,20190815,7/,20445476,-121.89253:+37.33943:0001,"{'type': 'MultiPolygon', 'coordinates': [[[[-1...",both


In [182]:
sc_projects_parcels.shape

(5146, 52)

In [183]:
sc_projects_parcels.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5146 entries, 0 to 5145
Data columns (total 52 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   mtc_id            5146 non-null   object  
 1   apn_x             5020 non-null   object  
 2   mtc_apn_fmt       4791 non-null   object  
 3   mtc_mapped        5146 non-null   bool    
 4   mtc_geocode_type  5146 non-null   object  
 5   objectid          3850 non-null   float64 
 6   apn_y             3850 non-null   object  
 7   apn2              0 non-null      float64 
 8   state             3850 non-null   object  
 9   county            3850 non-null   object  
 10  fips              3850 non-null   object  
 11  sit_hse_nu        2659 non-null   object  
 12  sit_dir           285 non-null    object  
 13  sit_str_na        2702 non-null   object  
 14  sit_str_sf        2580 non-null   object  
 15  sit_full_s        2702 non-null   object  
 16  sit_city          2741 n

In [184]:
cols = ['mtc_id','mtc_apn_fmt','xcoord','ycoord','_merge','mtc_geocode_type']
geocode_types = ['APPROXIMATE','GEOMETRIC_CENTER','NOT_GEOCODED']
sc_projects_parcels_sub = sc_projects_parcels[cols][(sc_projects_parcels['_merge'] == 'both') & 
                                                    (sc_projects_parcels['mtc_geocode_type'].isin(geocode_types))].copy()

In [185]:
sc_projects_parcels_sub

,mtc_id,mtc_apn_fmt,xcoord,ycoord,_merge,mtc_geocode_type
10,40a126ed-5940-4805-933f-abfb49366f68,41909026,-121.929,37.2575,both,APPROXIMATE
38,0d04ca96-76db-4796-8c06-948a99a25ec5,30944075,-122.027,37.3380,both,APPROXIMATE
39,f1a83e3d-f53e-47ab-ad2d-871491739dc7,30944074,-122.027,37.3382,both,APPROXIMATE
40,9c5fccc8-17b6-4b92-a603-ba3a20be767b,30944073,-122.027,37.3382,both,APPROXIMATE
41,594d7ac7-1bd3-40e7-aab9-ebd90e74b47e,30944072,-122.028,37.3382,both,APPROXIMATE
...,...,...,...,...,...,...
5076,2652a2d8-c820-49f6-9627-15cb78dde147,80854031,-121.595,36.9971,both,GEOMETRIC_CENTER
5077,00cc31ba-908a-49af-9810-6c0bd43e6b8f,80854030,-121.595,36.9971,both,GEOMETRIC_CENTER
5078,ec07d179-efd5-4c76-857a-c90032cf9386,80854029,-121.595,36.9972,both,GEOMETRIC_CENTER
5079,1b5c6db6-ac82-4d04-a649-1c35f6a4585a,80854028,-121.595,36.9974,both,GEOMETRIC_CENTER


In [186]:
sc_projects_parcels_sub[sc_projects_parcels_sub['mtc_geocode_type'] == 'NOT_GEOCODED'].shape

(32, 6)

In [187]:
#Create geodataframe from lat long coords
sc_geo = gp.GeoDataFrame(sc_projects_parcels_sub,
                                  geometry=gp.points_from_xy(sc_projects_parcels_sub['xcoord'],
                                                             sc_projects_parcels_sub['ycoord']),
                                  crs="EPSG:4326")

In [189]:
sc_geo.head(5)

,mtc_id,mtc_apn_fmt,xcoord,ycoord,_merge,mtc_geocode_type,geometry
10,40a126ed-5940-4805-933f-abfb49366f68,41909026,-121.929,37.2575,both,APPROXIMATE,POINT (-121.92900 37.25750)
38,0d04ca96-76db-4796-8c06-948a99a25ec5,30944075,-122.027,37.3380,both,APPROXIMATE,POINT (-122.02700 37.33800)
39,f1a83e3d-f53e-47ab-ad2d-871491739dc7,30944074,-122.027,37.3382,both,APPROXIMATE,POINT (-122.02700 37.33820)
40,9c5fccc8-17b6-4b92-a603-ba3a20be767b,30944073,-122.027,37.3382,both,APPROXIMATE,POINT (-122.02700 37.33820)
41,594d7ac7-1bd3-40e7-aab9-ebd90e74b47e,30944072,-122.028,37.3382,both,APPROXIMATE,POINT (-122.02800 37.33820)


In [190]:
sc_geo['mtc_wkt'] = sc_geo['geometry'].map(lambda x: x.wkt)

In [192]:
#Rename columns to match final schema
sc_geo.rename(columns={'xcoord':'mtc_long','ycoord':'mtc_lat'},inplace=True)

In [193]:
#Update mtc_mapped column and set to True
sc_geo['mtc_mapped'] = True

In [195]:
#Update mtc_geocode_type and set to APN_MATCH
sc_geo['mtc_geocode_type'] = 'APN_MATCH'

In [196]:
sc_geo.head(5)

,mtc_id,mtc_apn_fmt,mtc_long,mtc_lat,_merge,mtc_geocode_type,geometry,mtc_wkt,mtc_mapped
10,40a126ed-5940-4805-933f-abfb49366f68,41909026,-121.929,37.2575,both,APN_MATCH,POINT (-121.92900 37.25750),POINT (-121.929 37.2575),True
38,0d04ca96-76db-4796-8c06-948a99a25ec5,30944075,-122.027,37.3380,both,APN_MATCH,POINT (-122.02700 37.33800),POINT (-122.027 37.338),True
39,f1a83e3d-f53e-47ab-ad2d-871491739dc7,30944074,-122.027,37.3382,both,APN_MATCH,POINT (-122.02700 37.33820),POINT (-122.027 37.3382),True
40,9c5fccc8-17b6-4b92-a603-ba3a20be767b,30944073,-122.027,37.3382,both,APN_MATCH,POINT (-122.02700 37.33820),POINT (-122.027 37.3382),True
41,594d7ac7-1bd3-40e7-aab9-ebd90e74b47e,30944072,-122.028,37.3382,both,APN_MATCH,POINT (-122.02800 37.33820),POINT (-122.028 37.3382),True


In [197]:
#Upsert updated records to socrata
final_columns = ['mtc_id','mtc_apn_fmt','mtc_long','mtc_lat','mtc_mapped','mtc_geocode_type','mtc_wkt']
upsert_df_socrata(sc_geo[final_columns],apr_id)

upserting data to Socrata in 1 chunks of 309 rows each
upserted chunk 0
data upserted to briv-ikjp
took 1.9361 seconds
